In [1]:
class Option():
    
    def __init__(self):
        self.dataroot= 'D:/VILAB/datasets/for_inpainting/train/RAFDB' #image dataroot
        self.maskroot= 'D:/VILAB/datasets/for_inpainting/mask/mask_224'#mask dataroot
        self.batchSize= 1# Need to be set to 1
        self.fineSize=224 # image size
        self.input_nc=3  # input channel size for first stage
        self.input_nc_g=6 # input channel size for second stage
        self.output_nc=3# output channel size
        self.ngf=64 # inner channel
        self.ndf=64# inner channel
        self.which_model_netD='basic' # patch discriminator
        self.which_model_netF='feature'# feature patch discriminator
        self.which_model_netG='unet_csa'# seconde stage network
        self.which_model_netP='unet_256'# first stage network
        self.triple_weight=1
        self.name='irregular_mask_inpainting'
        self.n_layers_D='3' # network depth
        self.gpu_ids=[0]
        self.model='csa_net'
        self.checkpoints_dir='checkpoints/' #
        self.norm='instance'
        self.fixed_mask=1
        self.use_dropout=False
        self.init_type='normal'
        self.mask_type='random'
        self.lambda_A=100
        self.threshold=5/16.0
        self.stride=1
        self.shift_sz=1 # size of feature patch
        self.mask_thred=1
        self.bottleneck=512
        self.gp_lambda=10.0
        self.ncritic=5
        self.constrain='MSE'
        self.strength=1
        self.init_gain=0.02
        self.cosis=1
        self.gan_type='lsgan'
        self.gan_weight=0.2
        self.overlap=4
        self.skip=0
        self.display_freq=1000
        self.print_freq=50
        self.save_latest_freq=5000
        self.save_epoch_freq=2
        self.continue_train=True
        self.epoch_count=1
        self.phase='train'
        self.which_epoch='20'
        self.niter=20
        self.niter_decay=100
        self.beta1=0.5
        self.lr=0.0002
        self.lr_policy='lambda'
        self.lr_decay_iters=50
        self.isTrain=True

In [2]:
import time
from util.data_load import Data_load
from models.models import create_model
import torch
import os
import torchvision
from torch.utils import data
import torchvision.transforms as transforms
opt = Option()
transform_mask = transforms.Compose(
    [transforms.Resize((256,256)),
     transforms.ToTensor(),
    ])
transform = transforms.Compose(
    [#transforms.RandomHorizontalFlip(),
     transforms.Resize((256,256)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)])

dataset_train = Data_load(opt.dataroot, opt.maskroot, transform, transform_mask)
iterator_train = (data.DataLoader(dataset_train, batch_size=1,shuffle=True))
print(len(dataset_train))
model = create_model(opt)
total_steps = 0

D:\anaconda\envs\GAN\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


12271
csa_net


D:\anaconda\envs\GAN\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
D:\anaconda\envs\GAN\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


initialize network with normal


D:\VILAB\CSA_idea\models\networks.py:54: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, 0.0, gain)
D:\VILAB\CSA_idea\models\networks.py:64: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias.data, 0.0)


initialize network with normal
initialize network with normal
initialize network with normal
Loading pre-trained network!
---------- Networks initialized -------------
model [CSAModel] was created


In [3]:
iter_start_time = time.time()
save_dir = "D:/VILAB/CSA_fer_data/result"
for epoch in range(opt.epoch_count, opt.niter + opt.niter_decay + 1):   
    epoch_start_time = time.time()
    epoch_iter = 0

#     image, mask, gt = [x.cuda() for x in next(iterator_train)]
    for image, mask in (iterator_train):
        image=image.cuda()
        mask=mask.cuda()
        mask=mask[0][0]
        mask=torch.unsqueeze(mask,0)
        mask=torch.unsqueeze(mask,1)
        mask=mask.byte()
        total_steps += opt.batchSize
        epoch_iter +=opt.batchSize
        model.set_input(image,mask) # it not only sets the input data with mask, but also sets the latent mask.
        model.set_gt_latent()
        model.optimize_parameters()
        break
        if total_steps %opt.display_freq== 0:
            real_A,real_B,fake_B=model.get_current_visuals()
            #real_A=input, real_B=ground truth fake_b=output
            pic = (torch.cat([real_A, real_B,fake_B], dim=0) + 1) / 2.0
            torchvision.utils.save_image(pic, '%s/Epoch_(%d)_(%dof%d).jpg' % (
            save_dir, epoch, total_steps + 1, len(dataset_train)), nrow=2)
        if total_steps %1== 0:
            errors = model.get_current_errors()
            t = (time.time() - iter_start_time) / opt.batchSize
            print(errors)
    break
    if epoch % opt.save_epoch_freq == 0:
        print('saving the model at the end of epoch %d, iters %d' %(epoch, total_steps))
        model.save(epoch)

    print('End of epoch %d / %d \t Time Taken: %d sec' %
            (epoch, opt.niter + opt.niter_decay, time.time() - epoch_start_time))

    model.update_learning_rate()


D:\VILAB\CSA_idea\models\CSA.py:121: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cuda\Indexing.cu:1240.)
  self.input_A.narrow(1,0,1).masked_fill_(self.mask_global, 2*123.0/255.0 - 1.0)
D:\VILAB\CSA_idea\models\CSA.py:122: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cuda\Indexing.cu:1240.)
  self.input_A.narrow(1,1,1).masked_fill_(self.mask_global, 2*104.0/255.0 - 1.0)
D:\VILAB\CSA_idea\models\CSA.py:123: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\nativ

torch.Size([1, 512, 32, 32])
torch.Size([512, 32, 32])
torch.Size([1, 1024, 32, 32])
tensor([[[[-2.3689e+01, -5.7573e+01, -9.8838e+01,  ..., -3.5735e+01,
           -2.2026e+01, -1.4230e+01],
          [-5.2492e+00, -4.0623e+01, -8.1754e+01,  ..., -2.9330e+01,
           -3.2100e+01, -1.8822e+01],
          [ 1.8030e+01, -3.1485e+00, -2.6861e+01,  ..., -2.2342e+01,
           -2.8734e+01, -2.4852e+01],
          ...,
          [-3.2135e+00, -2.4895e+01, -3.7998e+01,  ..., -1.8315e+01,
           -7.2629e+01, -9.1551e+01],
          [-1.8162e+00, -8.6395e+00, -3.6550e+00,  ..., -4.5231e+01,
           -4.8756e+01, -7.1420e+01],
          [-8.7789e+00, -1.2857e+01, -6.5494e+00,  ..., -3.2747e+01,
           -6.8454e+01, -6.5551e+01]],

         [[ 6.9078e-01, -5.7994e+00, -9.5615e+00,  ..., -1.1897e+01,
           -3.9974e+00, -1.6838e+01],
          [-5.2018e+01, -8.6139e+01, -1.0573e+02,  ..., -2.6175e+01,
           -1.4434e+01, -2.2894e+01],
          [-2.8936e+01, -4.0475e+01, -5.15